In [1]:
from astropy.coordinates import SkyCoord
from astropy import units
import numpy as np
import pandas as pd
import urllib.parse
from astropy.table import Table
from astropy.coordinates import angular_separation

In [2]:
globalClusters = np.loadtxt(r"data/globalTestFormat.csv",delimiter=',',dtype=str)
globalNames = globalClusters.T[0][3:]
globalNames = np.char.strip(globalNames)
#dictionaries of globals, name: property
globalRAs = dict(zip(globalNames,globalClusters.T[8][3:]))
globalDECs = dict(zip(globalNames,globalClusters.T[9][3:]))
globalZs = dict(zip(globalNames,globalClusters.T[10][3:]))

profileClusters = np.loadtxt(r"data\ACCEPT2_K0_and_z.csv",delimiter=',',dtype=str)
profileNames = profileClusters.T[1][1:]
profileNames = np.char.strip(profileNames)
#dictionaries of profiles, name: property
profileRAs = dict(zip(profileNames,profileClusters.T[18][1:]))
profileDECs = dict(zip(profileNames,profileClusters.T[19][1:]))
profileZs = dict(zip(profileNames,profileClusters.T[12][1:]))
profileC500s = dict(zip(profileNames,profileClusters.T[13][1:]))

#combine the list of names
allNames = np.concatenate((globalNames,profileNames))
allNames=np.unique(allNames)
allNames.sort()

#initialize empty table
table = np.zeros((len(allNames),8),dtype=object)

#set first column to all accept2.0 names
table[:,0]=allNames

In [3]:
#populates the table with whether or not the cluster has global properties, profiles, and morphology
for i in range(len(allNames)):
    name = allNames[i]
    if name in globalNames:
        table[i,4]='y'
        table[i,1]=globalRAs[name]
        table[i,2]=globalDECs[name]
        table[i,3]=globalZs[name]
    else:
        table[i,4]='n'  
    if name in profileNames:
        linkName = urllib.parse.quote(name) #ceratin symbols, notably + in this case, need special encoding to be parsed as part of the string by php get
        table[i,5]='<a href="/clusterfigure?name=' + linkName + '">y</a>' #adds link to cluster page
        coords = SkyCoord(ra=profileRAs[name], dec=profileDECs[name], unit=(units.hourangle, units.degree))
        ra,dec=coords.to_string(precision=5).split(" ")
        table[i,1]=ra
        table[i,2]=dec
        table[i,3]=profileZs[name]
        if profileC500s[name] != '-': #morphology test
            table[i,6]='y'
        else:
            table[i,6]='n'
    else:
        table[i,5]='n'
        table[i,6]='n'


#checks if the ACCEPT2 cluster was in ACCEPT1
allRAs = np.radians(table[:,1].astype('float'))
allDECs = np.radians(table[:,2].astype('float'))

ACCEPT1 = Table.read('data/ACCEPT1.txt',format='ascii')

unassignedClusters = []

table[:,7]='n'
for i in range(len(ACCEPT1)):
    coords = SkyCoord(ra=ACCEPT1[i][1], dec=ACCEPT1[i][2], unit=(units.hourangle, units.degree))
    ra,dec=coords.to_string(unit=units.rad, precision=5).split(" ")
    sep = np.degrees(angular_separation(float(ra),float(dec),allRAs,allDECs))
    if min(sep)<0.1: #makes sure the number is within reason
        if table[np.argmin(sep),7]=='y':
            print('Duplicate Warning')
        table[np.argmin(sep),7]='y'
    else:
        unassignedClusters.append(ACCEPT1[i][0])
        print(ACCEPT1[i][0], table[np.argmin(sep),0]) #sanity check to compare names


#export
dfTable = pd.DataFrame(table,columns=['Cluster Name', 'RA', 'DEC', 'z', 'Global Properties', 'Profiles', 'Morphology', 'In ACCEPT1.0'],index=None) #set up table as pandas dataframe
htmlTable = dfTable.to_html('cluster_pointers.html',index=False,escape=False) #exports table to html table

3C_28.0 ZwCl_0040.8+2404
3C_388 MACS_J1824.3+4309
ABELL_133 ABELL_0141
ABELL_426 WBL_088
ABELL_520 MCXC_J0443.1+0210
ABELL_521 MCXC_J0510.7-0801
ABELL_754 Hydra_A
ABELL_1060 ABELL_3444
ABELL_1995 WHL_J135949.5+623047
ABELL_2029 MKW_03s
ABELL_2052 MKW_03s
ABELL_2063 MKW_03s
ABELL_2065 ABELL_2069
ABELL_2199 ABELL_2187
ABELL_2420 ABELL_2426
ABELL_2462 ABELL_2485
ABELL_2554 ABELL_2556
ABELL_2589 ABELL_2626
ABELL_3084 Fornax_Cluster
ABELL_3112 SPT-CLJ0310-4647
ABELL_3266 SPT-CLJ0426-5455
ABELL_3528S ABELL_3528B
ABELL_3822 SPT-CLJ2145-5644
ABELL_4059 ABELL_2717
ABELL_S0405 MCXC_J0303.7-7752
AC_114 ABELL_3880
CID_0072 NSC_J174715+451155
CYGNUS_A ABELL_2319
M49 MCXC_J1234.2+0947
M87 MESSIER_089
MACS_J0257.1-2325 ABELL_0402
NGC_0507 ABELL_0262
NGC_5813 WHL_J150407.5-024816
NGC_5846 WHL_J150407.5-024816
RBS_0533 MCXC_J0437.1+0043
RX_J0647.7+7015 ABELL_0562
RX_J1539.5-8335 ABELL_4023
RX_J2129.6+0005 MACS_J2135-0102
UGC_03957 ABELL_0576
ZWCL_1953 ABELL_0697
